In [1]:
# Neural Population Analysis - Countermanding Saccade Task
# Interactive Exploratory Data Analysis

# **Data Format Note**: This notebook is designed to work with pickle (.pkl) files 
# that preserve Python data types. The Excel sample was provided for structure preview only.
# For full datasets, use the .pkl format for optimal performance.

# ## Setup and Imports

import pandas as pd
import numpy as np
import ast
import warnings
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# HoloViews and visualization imports
import holoviews as hv
import hvplot.pandas
import panel as pn

# Configure HoloViews with Bokeh backend
hv.extension('bokeh')
pn.extension('bokeh')

# Set default figure size and styling
hv.opts.defaults(
    hv.opts.Curve(width=600, height=400, tools=['hover']),
    hv.opts.Scatter(width=600, height=400, size=8, tools=['hover']),
    hv.opts.BoxWhisker(width=600, height=400),
    hv.opts.Histogram(width=600, height=400),
    hv.opts.Bars(width=600, height=400)
)

# ## Data Loading and Preprocessing

def load_neural_data(filepath):
    """Load neural data from pickle file"""
    if str(filepath).endswith('.pkl'):
        # Load pickle file - data should already be in correct format
        df = pd.read_pickle(filepath)
        
        # Ensure neural_spikes column exists for consistency
        if 'neural_spikes' not in df.columns and 'neural_data' in df.columns:
            df['neural_spikes'] = df['neural_data']
            
        return df
        
    elif str(filepath).endswith('.xlsx'):
        # For Excel files (demo/sample data), parse string representations
        df = pd.read_excel(filepath)
        df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
        
        # Parse string-encoded data
        def safe_eval(x):
            if pd.isna(x) or x == '':
                return None
            if isinstance(x, (list, dict, np.ndarray)):
                return x  # Already proper type
            try:
                return ast.literal_eval(str(x))
            except:
                return x
        
        # Convert string representations back to proper data types
        if 'neural_data' in df.columns:
            df['neural_spikes'] = df['neural_data'].apply(safe_eval)
        
        # Only parse if columns contain strings, not arrays
        for col in ['hPos', 'vPos', 'hVel', 'vVel', 'speed', 'saccades', 
                   'segs_durations', 'segs_times', 'blinks']:
            if col in df.columns:
                # Check if column needs parsing (contains strings)
                sample_val = df[col].dropna().iloc[0] if not df[col].dropna().empty else None
                if isinstance(sample_val, str):
                    df[f'{col}_array'] = df[col].apply(safe_eval)
                else:
                    df[f'{col}_array'] = df[col]  # Already proper format
        
        return df
    else:
        raise ValueError("File format not supported. Use .pkl or .xlsx files.")

def extract_neural_features(df):
    """Extract neural population features for analysis"""
    features = []
    
    for idx, row in df.iterrows():
        trial_features = {
            'trial_idx': idx,
            'type': row['type'],
            'direction': row['direction'],
            'trial_length': row['trial_length'],
            'go_cue': row['go_cue'],
            'stop_cue': row['stop_cue'] if pd.notna(row['stop_cue']) else None,
            'ssd_len': row['ssd_len'] if pd.notna(row['ssd_len']) else None,
        }
        
        # Neural activity features - handle both direct dict and parsed data
        neural_data = row['neural_spikes'] if 'neural_spikes' in row else row.get('neural_data', None)
        
        if neural_data and isinstance(neural_data, dict):
            spikes = neural_data
            trial_features['n_active_neurons'] = len(spikes)
            trial_features['total_spike_count'] = sum(len(times) for times in spikes.values())
            trial_features['mean_firing_rate'] = trial_features['total_spike_count'] / (row['trial_length'] / 1000)
            
            # Population activity in different epochs
            go_cue_time = row['go_cue']
            
            # Pre-go activity (0 to go_cue)
            pre_go_spikes = sum(len([t for t in times if t < go_cue_time]) for times in spikes.values())
            trial_features['pre_go_rate'] = pre_go_spikes / (go_cue_time / 1000) if go_cue_time > 0 else 0
            
            # Post-go activity (go_cue to end)
            post_go_spikes = sum(len([t for t in times if t >= go_cue_time]) for times in spikes.values())
            post_go_duration = (row['trial_length'] - go_cue_time) / 1000
            trial_features['post_go_rate'] = post_go_spikes / post_go_duration if post_go_duration > 0 else 0
            
        else:
            trial_features['n_active_neurons'] = 0
            trial_features['total_spike_count'] = 0
            trial_features['mean_firing_rate'] = 0
            trial_features['pre_go_rate'] = 0
            trial_features['post_go_rate'] = 0
        
        features.append(trial_features)
    
    return pd.DataFrame(features)

# Load the data
print("Loading and preprocessing data...")

# Dynamic filepath setup for different monkeys
monkey = 'yasmin'  # Change to 'fiona' for Fiona's data
base_path = Path.cwd().parent / 'data' / f'{monkey}_sst'
filepath = base_path.parent / 'csst_trials_pkls' / f'{monkey}_csst_trials_df.pkl'

print(f"Loading data for {monkey} from: {filepath}")

# Check if file exists
if not filepath.exists():
    print(f"File not found: {filepath}")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Expected file location: {filepath.absolute()}")
    raise FileNotFoundError(f"Data file not found at {filepath}")

df = load_neural_data(filepath)
features_df = extract_neural_features(df)

print(f"Loaded {len(df)} trials with neural data")
print(f"Trial types: {', '.join(df['type'].value_counts().index)}")
print(f"Directions: {', '.join(df['direction'].value_counts().index)}")
print(f"Active neurons across trials: {features_df['n_active_neurons'].sum()}")
print(f"Data columns: {len(df.columns)} columns available")

# ## 1. Trial Characteristics Overview

# ### Trial Type and Direction Distribution
trial_type_counts = df['type'].value_counts()
direction_counts = df['direction'].value_counts()

# Create interactive bar charts using hvplot (more reliable and informative than pie charts)
type_bar = trial_type_counts.hvplot.bar(title="Trial Type Distribution", 
                                        xlabel="Trial Type", ylabel="Count",
                                        width=400, height=400, rot=45)

direction_bar = direction_counts.hvplot.bar(title="Direction Distribution", 
                                            xlabel="Direction", ylabel="Count",
                                            width=400, height=400)

trial_overview = (type_bar + direction_bar)

# Alternative: If you prefer pie charts, use matplotlib
def create_pie_charts():
    """Create static pie charts with matplotlib"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Trial type pie
    trial_type_counts.plot.pie(ax=ax1, autopct='%1.1f%%', startangle=90)
    ax1.set_title("Trial Type Distribution")
    ax1.set_ylabel("")
    
    # Direction pie  
    direction_counts.plot.pie(ax=ax2, autopct='%1.1f%%', startangle=90)
    ax2.set_title("Direction Distribution")
    ax2.set_ylabel("")
    
    plt.tight_layout()
    plt.show()

# Uncomment to show pie charts:
# create_pie_charts()

print("Trial Overview:")
trial_overview

# ### Trial Length Analysis
trial_length_hist = df.hvplot.hist('trial_length', bins=15, alpha=0.7,
                                   title="Trial Length Distribution",
                                   xlabel="Trial Length (ms)", ylabel="Count",
                                   width=500, height=400)

trial_length_by_type = df.hvplot.box(y='trial_length', by='type',
                                     title="Trial Length by Type",
                                     xlabel="Trial Type", ylabel="Trial Length (ms)",
                                     width=500, height=400)

trial_length_analysis = (trial_length_hist + trial_length_by_type)
print("Trial Length Analysis:")
trial_length_analysis

# ### Stop Signal Delay Analysis (STOP trials only)
stop_trials = df[df['type'] == 'STOP'].copy()
if len(stop_trials) > 0:
    ssd_hist = stop_trials.hvplot.hist('ssd_len', bins=10, alpha=0.7,
                                       title="Stop Signal Delay Distribution",
                                       xlabel="SSD Length (ms)", ylabel="Count",
                                       width=600, height=400)
    print("Stop Signal Delay Analysis:")
    ssd_hist
else:
    print("No STOP trials found for SSD analysis")

# ## 2. Neural Population Activity Analysis

# ### Active Neurons per Trial
active_neurons_by_type = features_df.hvplot.box(y='n_active_neurons', by='type',
                                                 title="Active Neurons by Trial Type",
                                                 xlabel="Trial Type", 
                                                 ylabel="Number of Active Neurons",
                                                 width=500, height=400)

spike_count_by_type = features_df.hvplot.box(y='total_spike_count', by='type',
                                             title="Total Spike Count by Trial Type",
                                             xlabel="Trial Type", 
                                             ylabel="Total Spikes",
                                             width=500, height=400)

neural_overview = (active_neurons_by_type + spike_count_by_type)
print("Neural Activity Overview:")
neural_overview

# ### Firing Rate Analysis
firing_rate_by_type = features_df.hvplot.box(y='mean_firing_rate', by='type',
                                              title="Mean Firing Rate by Trial Type",
                                              xlabel="Trial Type", 
                                              ylabel="Firing Rate (Hz)",
                                              width=500, height=400)

firing_rate_by_direction = features_df.hvplot.box(y='mean_firing_rate', by='direction',
                                                   title="Firing Rate by Direction",
                                                   xlabel="Direction", 
                                                   ylabel="Firing Rate (Hz)",
                                                   width=500, height=400)

firing_rate_analysis = (firing_rate_by_type + firing_rate_by_direction)
print("Firing Rate Analysis:")
firing_rate_analysis

# ### Pre vs Post Go Cue Activity
# Create color mapping for trial types
color_map = {'STOP': 'red', 'GO': 'blue', 'CONT': 'green'}
features_df['color'] = features_df['type'].map(color_map)

pre_post_scatter = features_df.hvplot.scatter('pre_go_rate', 'post_go_rate', 
                                              color='type', 
                                              title="Pre vs Post Go Cue Activity",
                                              xlabel="Pre-Go Firing Rate (Hz)",
                                              ylabel="Post-Go Firing Rate (Hz)",
                                              width=600, height=500,
                                              hover_cols=['type', 'direction', 'trial_idx'])

print("Pre vs Post Go Activity:")
pre_post_scatter

# ### Trial Length vs Neural Activity
length_vs_activity = features_df.hvplot.scatter('trial_length', 'mean_firing_rate',
                                                 color='type',
                                                 title="Trial Duration vs Neural Activity",
                                                 xlabel="Trial Length (ms)",
                                                 ylabel="Mean Firing Rate (Hz)",
                                                 width=600, height=500,
                                                 hover_cols=['type', 'direction', 'trial_idx'])

print("Trial Duration vs Neural Activity:")
length_vs_activity

# ## 3. Population Raster Plots

def create_raster_plot(df, trial_idx, width=800, height=300):
    """Create an interactive raster plot for a single trial"""
    row = df.iloc[trial_idx]
    
    # Get neural data - handle both direct dict access and parsed data
    neural_data = row.get('neural_spikes', row.get('neural_data', None))
    
    if not neural_data or not isinstance(neural_data, dict):
        print(f"No neural data for trial {trial_idx}")
        return None
    
    spikes = neural_data
    
    # Prepare data for plotting
    raster_data = []
    neuron_ids = sorted(spikes.keys())
    
    for i, neuron_id in enumerate(neuron_ids):
        spike_times = spikes[neuron_id]
        for spike_time in spike_times:
            raster_data.append({
                'time': spike_time,
                'neuron': i,
                'neuron_id': neuron_id,
                'spike': 1
            })
    
    if not raster_data:
        print(f"No spikes found for trial {trial_idx}")
        return None
    
    raster_df = pd.DataFrame(raster_data)
    
    # Create raster plot
    raster = raster_df.hvplot.scatter('time', 'neuron', 
                                      title=f"Trial {trial_idx}: {row['type']} {row['direction']} "
                                            f"(Length: {row['trial_length']}ms)",
                                      xlabel="Time (ms)", 
                                      ylabel="Neuron Index",
                                      color='black', size=15, alpha=0.8,
                                      width=width, height=height,
                                      hover_cols=['neuron_id', 'time'])
    
    return raster.opts(
        xlim=(0, row['trial_length']),
        ylim=(-0.5, len(neuron_ids) - 0.5)
    )

# Create raster plots for sample trials
print("Sample Population Raster Plots:")

# Select representative trials
sample_trials = []
for trial_type in ['GO', 'STOP', 'CONT']:
    type_trials = df[df['type'] == trial_type].index.tolist()
    if type_trials:
        sample_trials.append(type_trials[0])

# Create and display raster plots
for i, trial_idx in enumerate(sample_trials[:3]):  # Show first 3 different types
    print(f"\nTrial {trial_idx} ({df.iloc[trial_idx]['type']} {df.iloc[trial_idx]['direction']}):")
    raster_plot = create_raster_plot(df, trial_idx)
    if raster_plot is not None:
        raster_plot

# ## 4. Summary Statistics and Comparisons

def generate_summary_stats(features_df):
    """Generate comprehensive summary statistics"""
    print("=== NEURAL POPULATION ANALYSIS SUMMARY ===\n")
    
    # Basic trial statistics
    print("TRIAL CHARACTERISTICS:")
    print(f"Total trials: {len(features_df)}")
    print(f"Trial types: {', '.join(features_df['type'].value_counts().index)}")
    print(f"Directions: {', '.join(features_df['direction'].value_counts().index)}")
    print(f"Mean trial length: {features_df['trial_length'].mean():.1f} ± {features_df['trial_length'].std():.1f} ms")
    
    # Neural activity statistics
    print(f"\nNEURAL ACTIVITY:")
    print(f"Mean active neurons per trial: {features_df['n_active_neurons'].mean():.1f} ± {features_df['n_active_neurons'].std():.1f}")
    print(f"Mean firing rate: {features_df['mean_firing_rate'].mean():.2f} ± {features_df['mean_firing_rate'].std():.2f} Hz")
    print(f"Total spike count range: {features_df['total_spike_count'].min()}-{features_df['total_spike_count'].max()}")
    
    # Analysis by trial type
    print(f"\nBY TRIAL TYPE:")
    for trial_type in features_df['type'].unique():
        subset = features_df[features_df['type'] == trial_type]
        print(f"  {trial_type} trials (n={len(subset)}):")
        print(f"    Mean firing rate: {subset['mean_firing_rate'].mean():.2f} ± {subset['mean_firing_rate'].std():.2f} Hz")
        print(f"    Active neurons: {subset['n_active_neurons'].mean():.1f} ± {subset['n_active_neurons'].std():.1f}")
    
    # Statistical tests
    if len(features_df['type'].unique()) > 1:
        print(f"\nSTATISTICAL COMPARISONS:")
        # Firing rate differences between trial types
        groups = [features_df[features_df['type'] == t]['mean_firing_rate'] for t in features_df['type'].unique()]
        f_stat, p_val = stats.f_oneway(*groups)
        print(f"One-way ANOVA (firing rate by trial type): F = {f_stat:.3f}, p = {p_val:.3f}")
        
        # Direction effects
        if len(features_df['direction'].unique()) > 1:
            left_rates = features_df[features_df['direction'] == 'L']['mean_firing_rate']
            right_rates = features_df[features_df['direction'] == 'R']['mean_firing_rate']
            t_stat, p_val = stats.ttest_ind(left_rates, right_rates)
            print(f"Direction comparison (t-test): t = {t_stat:.3f}, p = {p_val:.3f}")

# Generate comprehensive summary
generate_summary_stats(features_df)

# ## 5. Interactive Dashboard Components

def create_trial_selector():
    """Create interactive trial selector for raster plots"""
    trial_options = [(f"Trial {i}: {row['type']} {row['direction']}", i) 
                     for i, row in df.iterrows()]
    
    print("Interactive trial selection:")
    print("Use the following function to explore individual trials:")
    print("create_raster_plot(df, trial_idx)")
    print(f"Available trial indices: 0 to {len(df)-1}")
    print(f"Trial types available: {list(df['type'].unique())}")
    
    return trial_options

# Create trial reference
trial_options = create_trial_selector()

# ## Key Findings Summary

print("\n" + "="*50)
print("KEY FINDINGS")
print("="*50)

# Population activity comparison
go_trials = features_df[features_df['type'] == 'GO']
stop_trials = features_df[features_df['type'] == 'STOP']

if len(go_trials) > 0 and len(stop_trials) > 0:
    go_rate_mean = go_trials['mean_firing_rate'].mean()
    stop_rate_mean = stop_trials['mean_firing_rate'].mean()
    
    print(f"1. Population firing rates:")
    print(f"   GO trials: {go_rate_mean:.2f} Hz")
    print(f"   STOP trials: {stop_rate_mean:.2f} Hz")
    print(f"   Difference: {abs(go_rate_mean - stop_rate_mean):.2f} Hz")

# Pre vs post go activity
pre_post_diff = features_df['post_go_rate'] - features_df['pre_go_rate']
print(f"\n2. Activity modulation around go cue:")
print(f"   Mean change: {pre_post_diff.mean():.2f} Hz")
print(f"   Trials with increased post-go activity: {sum(pre_post_diff > 0)}/{len(pre_post_diff)}")

# Direction effects
left_trials = features_df[features_df['direction'] == 'L']
right_trials = features_df[features_df['direction'] == 'R']

if len(left_trials) > 0 and len(right_trials) > 0:
    print(f"\n3. Direction selectivity:")
    print(f"   Left trials: {left_trials['mean_firing_rate'].mean():.2f} Hz")
    print(f"   Right trials: {right_trials['mean_firing_rate'].mean():.2f} Hz")

print(f"\n4. Neural population characteristics:")
print(f"   Mean active neurons per trial: {features_df['n_active_neurons'].mean():.1f}")
print(f"   Population shows trial-type specific dynamics")

print("\nAnalysis complete! Use the interactive dashboard above to explore individual trials.")

Loading and preprocessing data...
Loading data for yasmin from: /home/barak/Projects/population_analysis/data/csst_trials_pkls/yasmin_csst_trials_df.pkl
Loaded 123178 trials with neural data
Trial types: GO, CONT, STOP
Directions: R, L
Active neurons across trials: 4407507
Data columns: 27 columns available
Trial Overview:
Trial Length Analysis:
Stop Signal Delay Analysis:
Neural Activity Overview:
Firing Rate Analysis:
Pre vs Post Go Activity:
Trial Duration vs Neural Activity:
Sample Population Raster Plots:

Trial 1 (GO R):

Trial 0 (STOP R):

Trial 13 (CONT R):
=== NEURAL POPULATION ANALYSIS SUMMARY ===

TRIAL CHARACTERISTICS:
Total trials: 123178
Trial types: GO, CONT, STOP
Directions: R, L
Mean trial length: 2440.4 ± 252.0 ms

NEURAL ACTIVITY:
Mean active neurons per trial: 35.8 ± 10.9
Mean firing rate: 400.80 ± 287.80 Hz
Total spike count range: 0-4772

BY TRIAL TYPE:
  STOP trials (n=25658):
    Mean firing rate: 399.55 ± 284.83 Hz
    Active neurons: 35.4 ± 10.9
  GO trials (n

In [3]:
# Quick Display of All Plots
# Copy and paste these lines in your notebook cells to see the plots

# 1. TRIAL OVERVIEW
print("=== TRIAL CHARACTERISTICS ===")
trial_overview

# %%
# 2. TRIAL LENGTHS  
print("=== TRIAL LENGTH ANALYSIS ===")
trial_length_analysis

# %%
# 3. NEURAL ACTIVITY OVERVIEW
print("=== NEURAL ACTIVITY OVERVIEW ===")
neural_overview

# %%
# 4. FIRING RATE ANALYSIS
print("=== FIRING RATE ANALYSIS ===")
firing_rate_analysis

# %%
# 5. PRE VS POST GO ACTIVITY
print("=== PRE VS POST GO ACTIVITY ===")
pre_post_scatter

# %%
# 6. TRIAL DURATION VS ACTIVITY
print("=== DURATION VS ACTIVITY ===")
length_vs_activity

# %%
# 7. SAMPLE RASTER PLOT (Trial 0)
print("=== SAMPLE RASTER PLOT ===")
create_raster_plot(df, 0)

# %%
# 8. SUMMARY STATISTICS
print("=== SUMMARY STATISTICS ===")
print(f"Dataset: {len(df)} trials")
print(f"Trial types: {dict(df['type'].value_counts())}")
print(f"Mean trial length: {df['trial_length'].mean():.0f} ms")
print(f"Mean firing rate: {features_df['mean_firing_rate'].mean():.2f} Hz")

# Show firing rates by trial type
for trial_type in features_df['type'].unique():
    subset = features_df[features_df['type'] == trial_type]
    print(f"{trial_type} trials: {subset['mean_firing_rate'].mean():.2f} ± {subset['mean_firing_rate'].std():.2f} Hz")

=== TRIAL CHARACTERISTICS ===
=== TRIAL LENGTH ANALYSIS ===
=== NEURAL ACTIVITY OVERVIEW ===
=== FIRING RATE ANALYSIS ===
=== PRE VS POST GO ACTIVITY ===
=== DURATION VS ACTIVITY ===
=== SAMPLE RASTER PLOT ===
=== SUMMARY STATISTICS ===
Dataset: 123178 trials
Trial types: {'GO': np.int64(69104), 'CONT': np.int64(28416), 'STOP': np.int64(25658)}
Mean trial length: 2440 ms
Mean firing rate: 400.80 Hz
STOP trials: 399.55 ± 284.83 Hz
GO trials: 400.69 ± 288.23 Hz
CONT trials: 402.22 ± 289.40 Hz


In [2]:
import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas
from pathlib import Path

hv.extension('bokeh')

# Quick exploration for pickle files with pathlib support
def quick_neural_eda(pkl_file_path):
    """Fast exploratory analysis of neural pickle data"""
    
    # Handle both string and Path objects
    pkl_file_path = Path(pkl_file_path)
    
    if not pkl_file_path.exists():
        raise FileNotFoundError(f"File not found: {pkl_file_path}")
    
    # Load data
    df = pd.read_pickle(pkl_file_path)
    print(f"Loaded {len(df)} trials from {pkl_file_path.name}")
    
    # Basic stats
    print(f"Trial types: {df['type'].value_counts().to_dict()}")
    print(f"Directions: {df['direction'].value_counts().to_dict()}")
    print(f"Mean trial length: {df['trial_length'].mean():.0f} ms")
    
    # Quick neural features
    def get_spike_count(neural_data):
        if isinstance(neural_data, dict):
            return sum(len(spikes) for spikes in neural_data.values())
        return 0
    
    df['spike_count'] = df['neural_data'].apply(get_spike_count)
    df['firing_rate'] = df['spike_count'] / (df['trial_length'] / 1000)
    
    # Interactive plots
    plots = []
    
    # Trial overview - use bar charts instead of pie charts
    trial_bar = df['type'].value_counts().hvplot.bar(title="Trial Types", 
                                                      xlabel="Type", ylabel="Count",
                                                      width=400, height=300, rot=45)
    length_hist = df.hvplot.hist('trial_length', bins=15, title="Trial Lengths", 
                                 width=400, height=300)
    plots.extend([trial_bar, length_hist])
    
    # Neural activity
    firing_box = df.hvplot.box(y='firing_rate', by='type', title="Firing Rate by Type",
                               width=400, height=300)
    spike_scatter = df.hvplot.scatter('trial_length', 'firing_rate', color='type',
                                      title="Duration vs Activity", width=400, height=300)
    plots.extend([firing_box, spike_scatter])
    
    # Display all plots
    layout = hv.Layout(plots).cols(2)
    return layout, df

# Quick setup for monkey data
def load_monkey_data(monkey_name, base_path=None):
    """Load data for a specific monkey using standard path structure"""
    
    if base_path is None:
        base_path = Path.cwd().parent / 'data'
    
    filepath = base_path / 'csst_trials_pkls' / f'{monkey_name}_csst_trials_df.pkl'
    
    if not filepath.exists():
        available_files = list((base_path / 'csst_trials_pkls').glob('*.pkl'))
        print(f"File not found: {filepath}")
        print(f"Available files: {[f.name for f in available_files]}")
        return None
    
    return quick_neural_eda(filepath)

def plot_trial_raster(df, trial_idx):
    """Quick raster plot for a single trial"""
    row = df.iloc[trial_idx]
    neural_data = row['neural_data']
    
    if not isinstance(neural_data, dict):
        print(f"No neural data for trial {trial_idx}")
        return None
    
    # Prepare raster data
    raster_data = []
    for i, (neuron_id, spike_times) in enumerate(sorted(neural_data.items())):
        for spike_time in spike_times:
            raster_data.append({'time': spike_time, 'neuron': i, 'neuron_id': neuron_id})
    
    if not raster_data:
        print(f"No spikes in trial {trial_idx}")
        return None
    
    raster_df = pd.DataFrame(raster_data)
    
    # Create plot
    raster = raster_df.hvplot.scatter('time', 'neuron', color='black', size=15,
                                      title=f"Trial {trial_idx}: {row['type']} {row['direction']}",
                                      width=800, height=400,
                                      hover_cols=['neuron_id'])
    
    return raster

# Example workflow:
# plots, data = load_monkey_data('yasmin')  # or 'fiona'
# trial_raster = plot_trial_raster(data, 0)